# DIY Analytics: Beyond Excel

This Jupyter Notebook was used to generate the data set
for the pre-conference exercise that is a companion
to the AALL 2021 session "DIY Analytics: Beyond Excel".

In [50]:
import csv
from random import choices, randrange, randint, shuffle
from datetime import date, timedelta

In [51]:
# Three offices. The code is written so that if 
# you wanted to generate a new data set with more offices,
# simply add them to this list.

offices = ['Chicago', 'Tallahassee', 'Honolulu']

# The weights are used when we randomly assign attorneys
# to the offices. The net effect of this is that the offices
# will be different sizes.

office_weights = [randint(0, 100) for i in range(0,len(offices))]

In [52]:
# 20 Practice groups. This list was produced by looking at the
# practice groups of several AmLaw 100 law firms. Like offices,
# this list can be made larger or smaller without affecting
# the rest of the code.

practice_groups = ['Antitrust', 'Business Litigation', 
                  'Restructuring', 'Employee Benefits', 'Energy', 
                  'Entertainment', 'Financial Transactions', 'International Arbitration',
                  'Government Regulation', 'Health Care', 'Insurance',
                  'Intellectual Property', 'White Collar Crime', 'Appeals',
                  'Labor', 'Mergers & Acquisitions', 'Private Equity',
                  'Real Estate', 'Securities Litigation', 'Tax']

# Like offices, practice groups are assigned
# randomly, but with weights so that some
# will be larger than others.

practice_group_weights = [randint(0, 100) for i in range(0, len(practice_groups))]

In [53]:
# Each attorney has a name, an office, a practice group, and a
# weight. The weight influences how often the ask questions. In my
# experience, some attorneys use the library's services more than
# others, and the weight will simulate that. 

# Researchers have an office and a weight, which affects how
# many requests they handle. 

# The names of the attorneys and researchers are selected randomly
# from a complete list of characters from Shakespeare's plays.

dramatis_personae = [] 
attorneys = []
researchers = []

print("starting...")

with open('characters.csv', encoding="ISO-8859-1") as infile:
    reader = csv.reader(infile)
    for row in reader:
        dramatis_personae.append(row[0])

i = 0
for r in choices(dramatis_personae, k=15):
    researchers.append([r, 
                       offices[i%3],
                       '',
                       randint(0, 100)])
    i += 1

for a in (set(dramatis_personae) - set([r[0] for r in researchers])):
    attorneys.append([a, 
                    choices(offices, office_weights)[0],
                    choices(practice_groups, practice_group_weights)[0],
                    randint(0, 100)])
        
print("done!")    
print("sample of attorneys:")
for i in range(0, 5):
    print(attorneys[i])
print("sample of researchers:")
for i in range(0, 5):
    print(researchers[i])


starting...
done!
sample of attorneys:
['Ladies', 'Tallahassee', 'Entertainment', 60]
['Sir Thomas Erpingham', 'Chicago', 'Real Estate', 79]
['George Seacoal', 'Tallahassee', 'Restructuring', 2]
['Lord', 'Tallahassee', 'Private Equity', 29]
['Cobbler', 'Tallahassee', 'Health Care', 89]
sample of researchers:
['Hal', 'Chicago', '', 30]
['Justice Silence', 'Tallahassee', '', 100]
['Paulina', 'Honolulu', '', 33]
['Romeo', 'Chicago', '', 30]
['Lord Berkeley', 'Tallahassee', '', 18]


In [54]:
# We randomly weight the days of the week, but hard code
# extra low values for the weekends

day_of_the_week = [randint(0, 10) for i in range(0, 7)]
day_of_the_week[5] = 3 # Saturday
day_of_the_week[6] = 3 # Sunday

In [55]:
# Holidays. The year of data we are analyzing is 2017,
# which started on a Sunday. This list includes the major US holidays,
# including New Year's Day, Presidents Day, etc. 

holidays = [0, 1, 15, 50, 148, 169, 184, 246, 314, 326, 327, 358]
holiday_weight = 1

In [56]:
# We pre-calculate the number of requests, per day, for
# one of the offices. The formula used will produce an upward trend 
# with a dramatic uptick near the beginning of the year.

upward = []

for i in range(0, 366):
    n = (1, -1)[(i-40)<0]
    x = (4.0 * pow(abs(i-40.0), 1.0/3)) * n + 30
    raw_number = randint(randint(0, int(.75 * x)), int(x))
    if i in holidays:
        raw_number = int(raw_number * holiday_weight)
    else:
        raw_number = int(raw_number * day_of_the_week[i % 7])
    upward.append(raw_number)


In [60]:
# Like above,  but this office will have a downward trend
# that almost moves to nothing

downward = []

from math import sqrt

for i in range(0, 366):
    x = 35 - sqrt(10 + (3*i))
    raw_number = randint(randint(0, int(.75 * x)), int(x))
    if i in holidays:
        raw_number = int(raw_number * holiday_weight)
    else:
        raw_number = int(raw_number * day_of_the_week[i % 7])
    downward.append(raw_number)

In [65]:
# lastly, the final office will have a basically flat trend

flat = []

for i in range(0, 366):
    raw_number = randint(randint(26, 35), 35) 
    if i in holidays:
        raw_number = int(raw_number * holiday_weight)
    else:
        raw_number = int(raw_number * day_of_the_week[i % 7])
    flat.append(raw_number) 

In [66]:
# This assigns the number of requests per day
# to each office. 

request_numbers = [upward, downward, flat]
shuffle(request_numbers)
random_offices = offices.copy()
shuffle(random_offices)

office_daily_request_numbers = {}

for i in range(0, len(random_offices)):
    office_daily_request_numbers[random_offices[i]] = request_numbers[i]

In [67]:
# Like offices and practice groups, this list of categories
# can be expanded or shrunk and everything else should work fine.

request_categories = ['Legal Research', 'Company Research', 'Other Research', 
                      'Books and Documents', 'Tech Support']

request_category_weights = [randint(0, 100) for i in range(0, len(request_categories))]


In [68]:
# This gives me 5 minutes increments for time spent

time_spent = [i for i in range(5, 121, 5)]
time_spent_weights = [randint(0, 100) for i in range(0, len(time_spent))]

In [69]:
# And resources. Unlike most of the other elements, this is intentionally
# weighted so that 'None' shows up more often than the other resources.

resources = ['Other', 'Lexis', 'Westlaw', 'Bloomberg Law', 'Print']
resource_weights = [50] + [randint(0, 25) for i in range(0, len(resources) - 1)]

In [70]:
# Now we generate the requests.csv file. First, a little prep work
# to make what comes later a little easier.

request_serial_number = randint(5, 10000)

january_1 = date(2017, 1, 1)

attorney_names = {}
attorney_weights = {}

for o in offices:
    attorney_names[o] = [a for a in attorneys if a[1] == o]
    attorney_weights[o] = [a[3] for a in attorneys if a[1] == o]

researcher_weights = [a[3] for a in researchers]

# the file should be of the form:
# serial number, date of request, attorney who made the request, attorney office,
# attorney practice group, 
# researcher, researcher's office, request category, time spent, resource used

with open("requests.csv", 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['Request Number', 'Date', 'Attorney', 'Attorney Office',
                     'Attorney Practice Group', 
                     'Researcher', 'Researcher Office', 'Request Category',
                     'Time Spent', 'Resource Used'])
    
    # Sample attorney
    # ['Hume', 'Tallahassee', 'Intellectual Property', 37]

    for day in range(0, 366):
        for office in offices:
            for request in range(0, office_daily_request_numbers[office][day]):
                request_serial_number += 1
                attorney = choices(attorney_names[office], attorney_weights[office])[0]
                researcher = choices(researchers, researcher_weights)[0]
                request_date = january_1 + timedelta(day)
                
                writer.writerow([request_serial_number,
                                request_date,
                                attorney[0],
                                attorney[1],
                                attorney[2],
                                researcher[0],
                                researcher[1],
                                choices(request_categories, request_category_weights)[0],
                                choices(time_spent, time_spent_weights)[0],
                                choices(resources, resource_weights)[0]])
